[PyCUDA](https://mathema.tician.de/software/pycuda/) is a Python wrapper around CUDA, NVidia's extension of C/C++ for GPUs.

There's also a [PyOpenCL](https://mathema.tician.de/software/pyopencl/) for the vendor-independent OpenCL standard.


Following shows the CUDA parallel thread hierarchy. CUDA executes kernels using a grid of blocks of threads. This figure shows the common indexing pattern used in CUDA programs using the CUDA keywords `gridDim.x` (the number of thread blocks), `blockDim.x` (the number of threads in each block), `blockIdx.x` (the index the current block within the grid), and `threadIdx.`x (the index of the current thread within the block).

<img src="assets/cuda_indexing.png">

Before you can use PyCuda, you have to import and initialize it:

In [1]:
import math
import numpy

import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

Note that you do not have to use `pycuda.autoinit` - initialization, context creation, and cleanup can also be performed manually, if desired

For this tutorial, we'll stick to something simple: we will write code to double each entry in array. To this end, we write the corresponding CUDA C code, and feed it into the constructor of a `pycuda.compiler.SourceModule`:

In [2]:
# generate a CUDA (C-ish) function that will run on the GPU; PROBLEM_SIZE is hard-wired
module = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

# pull "doublify" out as a Python callable
doublify = module.get_function("doublify")

In the following, we will create some data as numpy arrays on the CPU.

The next step in this and most other programs is to transfer data onto the device. In PyCuda, you will mostly transfer data from numpy arrays on the host. 

In [3]:
# create Numpy arrays on the CPU
a = numpy.random.randn(4,4).astype(numpy.float32)

#next, we need somewhere to transfer data to, so we need to allocate memory on the device:
a_gpu = cuda.mem_alloc(a.nbytes)

#as a last step, we need to transfer the data to the GPU:

cuda.memcpy_htod(a_gpu, a)


# define block/grid size for our problem: 4x4 block size
blockdim = (4,4,1)
#griddim = (int(math.ceil(PROBLEM_SIZE / 512.0)), 1, 1)

# copy the "driver.In" arrays to the GPU, run the 
#just_multiply(driver.Out(dest), driver.In(a), driver.In(b), block=blockdim, grid=griddim)
doublify(a_gpu, block=blockdim)

In [4]:
#Finally, we fetch the data back from the GPU and display it, together with the original a:

a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print (a_doubled)
print (a)
#doublify(cuda.InOut(a), block=blockdim)

[[-1.23081899 -1.60914695 -0.21349314 -0.58991599]
 [-1.53882289  3.44664907  1.44528294 -3.1512928 ]
 [-2.77189636  0.24643208 -1.41279054  0.12612192]
 [ 0.43340847 -3.97955871 -2.67981982  0.26493114]]
[[-0.61540949 -0.80457348 -0.10674657 -0.294958  ]
 [-0.76941144  1.72332454  0.72264147 -1.5756464 ]
 [-1.38594818  0.12321604 -0.70639527  0.06306096]
 [ 0.21670423 -1.98977935 -1.33990991  0.13246557]]


Now let's do that calculation of $\pi$.

In [6]:
PROBLEM_SIZE = int(1e6)

module2 = SourceModule("""
__global__ void mapper(float *dest)
{
  const int id = threadIdx.x + blockDim.x*blockIdx.x;
  const double x = 1.0 * id / %d;     // x goes from 0.0 to 1.0 in PROBLEM_SIZE steps
  if (id < %d)
    dest[id] = 4.0 / (1.0 + x*x);
}
""" % (PROBLEM_SIZE, PROBLEM_SIZE))

mapper = module2.get_function("mapper")
dest = numpy.empty(PROBLEM_SIZE, dtype=numpy.float32)
blockdim = (512, 1, 1)
griddim = (int(math.ceil(PROBLEM_SIZE / 512.0)), 1, 1)

mapper(cuda.Out(dest), block=blockdim, grid=griddim)

dest.sum() * (1.0 / PROBLEM_SIZE)  # correct for bin size

3.141594

We're doing the mapper (problem of size 1 million) on the GPU and the final sum (problem of size 1 million) on the CPU.

However, we want to do all the big data work on the GPU.

On the next slide is an algorithm that merges array elements with their neighbors in $\log_2(\mbox{million}) = 20$ steps.

In [7]:
module3 = SourceModule("""
__global__ void reducer(float *dest, int i)
{
  const int PROBLEM_SIZE = %d;
  const int id = threadIdx.x + blockDim.x*blockIdx.x;
  if (id %% (2*i) == 0  &&  id + i < PROBLEM_SIZE) {
    dest[id] += dest[id + i];
  }
}
""" % PROBLEM_SIZE)

blockdim = (512, 1, 1)
griddim = (int(math.ceil(PROBLEM_SIZE / 512.0)), 1, 1)

reducer = module3.get_function("reducer")

# Python for loop over the 20 steps to reduce the array
i = 1
while i < PROBLEM_SIZE:
    reducer(cuda.InOut(dest), numpy.int32(i), block=blockdim, grid=griddim)
    i *= 2

# final result is in the first element
dest[0] * (1.0 / PROBLEM_SIZE)

3.1415934999999999

The only problem now is that we're copying this `dest` array back and forth between the CPU and GPU. Let's fix that:

In [9]:
# allocate the array directly on the GPU, no CPU involved
dest_gpu = cuda.mem_alloc(PROBLEM_SIZE * numpy.dtype(numpy.float32).itemsize)

# do it again without "driver.InOut", which copies Numpy (CPU) to and from the GPU
mapper(dest_gpu, block=blockdim, grid=griddim)
i = 1
while i < PROBLEM_SIZE:
    reducer(dest_gpu, numpy.int32(i), block=blockdim, grid=griddim)
    i *= 2

# we only need the first element, so create a Numpy array with exactly one element
only_one_element = numpy.empty(1, dtype=numpy.float32)

# copy just that one element
cuda.memcpy_dtoh(only_one_element, dest_gpu)

print (only_one_element[0] * (1.0 / PROBLEM_SIZE))

3.1415935
